#Import cityscapes

In [ ]:
! mkdir /content/cityScapes

In [ ]:
! wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=d.a.vandendoel@students.uu.nl&password=Derkojo95!&submit=Login' https://www.cityscapes-dataset.com/login/
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1 -P /content/cityScapes
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3 -P /content/cityScapes

In [ ]:
! unzip /content/cityScapes/gtFine_trainvaltest.zip -d /content/cityScapes/annotations
! unzip /content/cityScapes/leftImg8bit_trainvaltest.zip -d /content/cityScapes/img

#Mount drive and import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow_addons

In [ ]:
!python -m pip install cityscapesscripts

In [ ]:
%cd /content/drive/MyDrive/Scriptie/python
import Generators
import Discriminators

In [ ]:
%cd /content
import cv2
import datetime
import itertools
import os
import PIL
import scipy
import sys
import time
import json

from cityscapesscripts.preparation.json2instanceImg import json2instanceImg
from cityscapesscripts.preparation.json2labelImg import json2labelImg
import numpy as np 
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.layers import SpectralNormalization
import matplotlib.pyplot as plt
from skimage import transform
from tqdm import tqdm
from random import randint, seed
from google.colab.patches import cv2_imshow
from imageio import imread
from tensorflow.keras import layers
from glob import glob
from IPython import display
from random import randint
from skimage.metrics import structural_similarity as SSIM
from collections import namedtuple


print(os.listdir("../content/"))

# Load images

In [ ]:
IM_HEIGHT = 256
IM_WIDTH = 512
BUFFER_SIZE = 2975
BATCH_SIZE = 1
OUTPUT_CHANNELS = 3
MASK_HEIGTH = 64
MASK_WIDTH = 32
LAMBDA = 100

In [ ]:
def load(image_file, channels):
  image = tf.io.read_file(image_file)
  image = tf.io.decode_png(image, channels = channels)
  image = tf.cast(image, tf.float32)

  return image

In [ ]:
def resize(input_image, height, width):
    image = tf.image.resize(
        input_image, 
        [height, width],
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
    )
  
    return image

In [ ]:
# Normalize values between -1 and +1
def normalize(image):
  image = image / 127.5 - 1
  return image

# Normalize values between 0 and 1 for image display
def denormalize(image):
  return image *0.5+0.5

# Create training dataset

In [ ]:
# Fill pandas dataframe with the paths to the files
def folder_to_pd(image_dir, semantics_dir):
  image_paths = {"train": [], "val": []}
  semantics_paths = {"train": [], "val": []}
  label_paths = {"train": [], "val": []}


  for data_type in ["train", "val"]:
    image_subdir = os.path.join(image_dir, data_type)
    for root, subdir, files in os.walk(image_subdir):
      subdir.sort()
      files.sort()
      if files:
        for file in files:
          image_paths[data_type].append(os.path.join(root, file))

    semantics_subdir = os.path.join(semantics_dir, data_type)
    for root, subdir, files in os.walk(semantics_subdir):
      subdir.sort()
      files.sort()
      if files:
        for i,file in enumerate(files):
          if file[-12:] == "labelIds.png":
            semantics_paths[data_type].append(os.path.join(root, file))
          if file[-5:] == ".json":
            label_paths[data_type].append(os.path.join(root, file))
  
  train_ds = pd.DataFrame(list(zip(image_paths["train"], semantics_paths["train"], label_paths["train"])), columns =['image', 'labelIds', 'json'])
  val_ds = pd.DataFrame(list(zip(image_paths["val"], semantics_paths["val"], label_paths["val"])), columns =['image', 'labelIds', 'json'])

  return train_ds, val_ds

#Function that determines whether there is a person present in the image
def flag_person(json_path):
  with open(json_path) as json_file:
    json_object = json.load(json_file)
  return any(element["label"] == "person" for element in json_object["objects"])

#function that filters out all the images without images in them
# if filter persons is true, get all images WITHOUT person. If False get all images WITH persons
def filter_ds(ds, filter_person = True):
  json_list = ds["json"]
  drop_rows = [i for i,element in enumerate(json_list) if flag_person(element) == filter_person]
  return ds.drop(drop_rows)

# convert pandas dataframe to tensorflow dataset
def pd_to_ds(pd_df):
  return tf.data.Dataset.from_tensor_slices((pd_df["image"], pd_df["labelIds"], pd_df["json"])) #returns dataset with image and labels for each element in the dataset

In [ ]:
def load_images_CS(info, mask):
  mask = load(mask, 0)
  mask = mask /255

  image_file, labels_file, json_file = info
  image = load(image_file, 3)
  image = resize(image, 256,512)
  masked_image = image * (1-mask)
  image = normalize(image)
  masked_image = normalize(masked_image)  

  labels = load(labels_file, 0)
  labels = resize(labels, 256,512)
  masked_labels = labels * (1-mask)

  labels = tf.cast(labels, tf.int32)
  masked_labels = tf.cast(masked_labels, tf.int32)

  #image = label_to_rgb(tf.expand_dims(labels, 0))
  #masked_image = label_to_rgb(tf.expand_dims(masked_labels, 0))

  #image = tf.squeeze(image, 0)
  #masked_image = tf.squeeze(masked_image, 0)

  return image, masked_image, labels, masked_labels, mask

In [ ]:
def load_images_person(image_file, labels_file, json_file):
  personID = 24
  riderID = 25
  
  labels = load(labels_file, 1)
  labels = resize(labels, 256,512)
  labels = tf.cast(labels, tf.int32)
    
  mask_person = tf.math.equal(labels, personID)
  mask_rider = tf.math.equal(labels, riderID)
  mask_person = tf.cast(mask_person, tf.int32)
  mask_rider = tf.cast(mask_rider, tf.int32)
  mask = mask_person + mask_rider
  masked_labels = labels * (1-mask)
  
  image = load(image_file, 3)
  image = resize(image, 256,512)
  mask = tf.cast(mask, tf.float32)
  masked_image = image * (1-mask)
  image = normalize(image)
  masked_image = normalize(masked_image)
  
  return image, masked_image, labels, masked_labels, mask

In [ ]:
#Create fine tuning datasets
tf_train, tf_val = folder_to_pd("/content/cityScapes/img/leftImg8bit", "/content/cityScapes/annotations/gtFine")

In [ ]:
# dataset with where each element is a tuple (path_to_image, path_to_labels) 
train_noPersons = pd_to_ds(filter_ds(tf_train))
val_noPersons = pd_to_ds(filter_ds(tf_val))

train_persons = pd_to_ds(filter_ds(tf_train, False))
val_persons = pd_to_ds(filter_ds(tf_val, False))

print("Training dataset WITHOUT persons has", train_noPersons.cardinality().numpy(), "elements.")
print("Validation dataset WITHOUT persons has", val_noPersons.cardinality().numpy(), "elements.")
print("")
print("Training dataset WITH persons has", train_persons.cardinality().numpy(), "elements.")
print("Validation dataset WITH persons has", val_persons.cardinality().numpy(), "elements.")

Zip mask and image data together.

In [ ]:
masks = tf.data.Dataset.list_files("/content/drive/MyDrive/Scriptie/masks/train/*")
masks_val = tf.data.Dataset.list_files("/content/drive/MyDrive/Scriptie/masks/val/*")

train_noPersons = tf.data.Dataset.zip((train_noPersons, masks))
val_noPersons = tf.data.Dataset.zip((val_noPersons, masks_val))

In [ ]:
finetune_train = train_noPersons.shuffle(train_noPersons.cardinality(), reshuffle_each_iteration=True)
finetune_train = finetune_train.map(load_images_CS, num_parallel_calls=tf.data.AUTOTUNE)
finetune_train = finetune_train.batch(BATCH_SIZE)
#finetune_train = finetune_train.cache("/content/temporary.tfcache")
finetune_train = finetune_train.prefetch(tf.data.AUTOTUNE)

finetune_val = val_noPersons.map(load_images_CS, num_parallel_calls=tf.data.AUTOTUNE)
finetune_val = finetune_val.batch(BATCH_SIZE)
finetune_val = finetune_val.prefetch(tf.data.AUTOTUNE)

In [ ]:
tf_val_persons = val_persons.map(load_images_person, num_parallel_calls=tf.data.AUTOTUNE)
tf_val_persons = tf_val_persons.batch(BATCH_SIZE)
tf_val_persons = tf_val_persons.prefetch(tf.data.AUTOTUNE)

In [ ]:
for step, (image, masked_image, labels, masked_labels, mask) in enumerate(tf_val_persons):
  if step == 396:
    plt.imshow(masked_image[0]*0.5+0.5)

# GAN

In [ ]:
generator = Generators.Generator(3)
discriminator = Discriminators.Discriminator()

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

def generator_loss( disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    # mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

#Training

Adjust the optimzer for the task of fine-tuning to get the best samples.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.5)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/Scriptie/ckpt/finetune/GAN'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_1e-4")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
checkpoint.restore("/content/drive/MyDrive/Scriptie/ckpt/GAN/ckpt_2e-5-1")

In [ ]:
def generate_images(model, test_input, tar, mask):  
  prediction = model(test_input, training=True)
  prediction = prediction*mask+tar*(1-mask)
  plt.figure(figsize=(20, 20))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(denormalize(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
@tf.function
def train_epoch(input_image, mask, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss
    
  
def getValAccLoss(input_image, mask, target): 
  gen_output = generator(input_image, training=True)
  gen_output = gen_output*mask+ target*(1-mask) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminator([input_image, target], training=True)
  disc_generated_output = discriminator([input_image, gen_output], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/finetune/GAN/'
summary_writer_train = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_TrainFT")
summary_writer_val = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_ValFT")

In [ ]:
def fit(train_ds, test_ds, epochs):
  example_target, example_input, label_test, label_masked_test, mask_test = next(iter(test_ds.take(1)))
  example_input, example_target, mask_test
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    generate_images(generator, example_input, example_target, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(target, input_image, labels, masked_labels, mask) in train_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epoch(input_image, mask, target)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]

      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)

    
    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(target, input_image, labels, masked_labels, mask) in test_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLoss(input_image, mask, target)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
#    if epoch % 5 == 0 & epoch !=0: 
#      checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
fit(finetune_train, finetune_val, 5)

In [ ]:
checkpoint.save("/content/drive/MyDrive/Scriptie/ckpt/finetune/GAN/GAN")

# Training SemGAN

## SemanticsGAN

In [ ]:
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (246, 35,232) ), #244,35,232
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ), #107 142 35
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

#convert rgb to grayscale values, this produces a unique grayscale value for each label
def rgb_gray(rgb):
  r, g ,b = rgb
  return 0.2989*r + 0.5870*g + 0.1140*b

#create dictionary with grayscale as key and id as value
color_id_list = []
color_id = {}
id_r = {}
id_g = {}
id_b = {}
for label in labels:
  key_list = tf.image.rgb_to_grayscale(tf.convert_to_tensor(label.color))
  key = int(rgb_gray(label.color))
  color_id_list.append((key_list, label.id))
  color_id[key] = label.id
  id_r[label.id] = label.color[0]
  id_g[label.id] = label.color[1]
  id_b[label.id] = label.color[2]
color_id[0] = 0
color_id[16] = 26 # grayscale value 16 has category "car"


#tf.image.rgb_to_grayscale
#convert dictionary to tf StaticHashTable
init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(color_id.keys())), tf.convert_to_tensor((list(color_id.values()))))
gray2label = tf.lookup.StaticHashTable(init,default_value= -100)

init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(id_r.keys())), tf.convert_to_tensor((list(id_r.values()))))
label2r =  tf.lookup.StaticHashTable(init,default_value= -100)

init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(id_g.keys())), tf.convert_to_tensor((list(id_g.values()))))
label2g =  tf.lookup.StaticHashTable(init,default_value= -100)

init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(id_b.keys())), tf.convert_to_tensor((list(id_b.values()))))
label2b =  tf.lookup.StaticHashTable(init,default_value= -100)

In [ ]:
def label_to_rgb(labeled_im):
  r = label2r.lookup(labeled_im)[:,:,:,0]
  g = label2g.lookup(labeled_im)[:,:,:,0]
  b = label2b.lookup(labeled_im)[:,:,:,0]

  image = tf.stack([r,g,b], axis = 3)
  image = tf.cast(image, tf.float32)
  image = normalize(image)

  return image

In [ ]:
def gen_to_label(gen_output):
  return tf.expand_dims(tf.math.argmax(gen_output, axis = -1, output_type = tf.dtypes.int32), -1)

def label_to_sparse(labels):
  return tf.squeeze(tf.one_hot(labels, depth = 35),3)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
  # Sparse Categorical Crossentropy
  CE_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(target, gen_output)
  total_gen_loss = gan_loss + CE_loss

  return total_gen_loss, gan_loss, CE_loss

def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
generator_semantics = Generators.GeneratorSemantics(IM_HEIGHT, IM_WIDTH)
discriminator_semantics = Discriminators.DiscriminatorSemantics(IM_HEIGHT, IM_WIDTH)

lr = 1e-5
generator_optimizer_semantics = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.5)
discriminator_optimizer_semantics = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.5)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting'
checkpoint_prefix = os.path.join(checkpoint_dir, "semantics", "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer_semantics,
                                 discriminator_optimizer=discriminator_optimizer_semantics,
                                 generator=generator_semantics,
                                 discriminator=discriminator_semantics)

In [ ]:
checkpoint.restore("/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting/finetune/ft_semantics-2")

##Finetune SemanticsGAN

In [ ]:
def generate_images(model, input, tar, mask):  
  prediction = model(input, training=True)
  plt.figure(figsize=(15, 15))

  prediction = tf.math.argmax(prediction, axis = -1, output_type = tf.dtypes.int32)
  prediction = label_to_rgb(tf.expand_dims(prediction, -1))
  
  input = tf.cast(input, tf.int32)
  input = label_to_rgb(input)


  display_list = [input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
@tf.function
def train_epoch(input_image, mask, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator_semantics(input_image, training=True)
    gen_output_argmax = gen_to_label(gen_output) #convert the 35 channel output of the generator to a 1 channel output with labels

    disc_real_output = discriminator_semantics([input_image, target], training=True)
    disc_generated_output = discriminator_semantics([input_image, gen_output_argmax], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  # from inpainted patches, count pixels that are not equal to the ground truth
  mask_size = tf.math.count_nonzero(mask)
  mask = tf.cast(mask, tf.int32)
  gen_acc = tf.math.count_nonzero(tf.math.not_equal(target*mask, gen_output_argmax*mask))/mask_size

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator_semantics.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminator_semantics.trainable_variables)

  generator_optimizer_semantics.apply_gradients(zip(generator_gradients, generator_semantics.trainable_variables))
  discriminator_optimizer_semantics.apply_gradients(zip(discriminator_gradients, discriminator_semantics.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc
    
  
def getValAccLoss(input_image, mask, target): 
  gen_output = generator_semantics(input_image, training=True)
  gen_output_argmax = gen_to_label(gen_output) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminator_semantics([input_image, target], training=True)
  disc_generated_output = discriminator_semantics([input_image, gen_output_argmax], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  mask_size = tf.math.count_nonzero(mask)
  mask = tf.cast(mask, tf.int32)
  gen_acc = tf.math.count_nonzero(tf.math.not_equal(target*mask, gen_output_argmax*mask))/mask_size

  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/SemanticInpainting/epoch/'
summary_writer_train = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Train")
summary_writer_val = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Val")

In [ ]:
def fit(train_ds, test_ds, epochs):
  #image, masked_image, labels, masked_labels, mask
  image_target, image_masked_test, label_test, masked_label_test, mask_test = next(iter(test_ds.take(1)))
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_CE_loss":0, "disc_loss":0, "gen_acc":0}
    generate_images(generator_semantics, masked_label_test, image_target, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(image, masked_image, label, masked_label, mask) in train_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc = train_epoch(masked_label, mask, label)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_CE_loss', metrics_dict["gen_CE_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_acc', metrics_dict["gen_acc"]/tf.cast(total_ims, tf.float64), step=epoch)


    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_CE_loss":0, "disc_loss":0, "gen_acc":0}
    print("Gathering validation set metrics...")
    for step,(image, masked_image, label, masked_label, mask) in test_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc = getValAccLoss(masked_label, mask, label)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_CE_loss', metrics_dict_val["gen_CE_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_acc', metrics_dict_val["gen_acc"]/tf.cast(total_ims_val, tf.float64), step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 10 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
fit(finetune_train, finetune_val, 5)

## SemGAN

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

def generator_loss( disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output)) # mean absolute error
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

In [ ]:
generatorSEMGAN = Generators.GeneratorSemGAN(OUTPUT_CHANNELS, IM_HEIGHT, IM_WIDTH)
discriminatorSEMGAN = Discriminators.DiscriminatorSemGAN(IM_HEIGHT, IM_WIDTH)

In [ ]:
lr = 2e-5
generator_optimizer = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.7)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.5)

In [ ]:
#checkpoint_dir_SEMGAN = '/content/drive/MyDrive/Scriptie/ckpt/SEMGAN/'
#checkpoint_prefix_SEMGAN = os.path.join(checkpoint_dir_SEMGAN, "ckpt", "sparse")
checkpoint_SEMGAN = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generatorSEMGAN,
                                 discriminator=discriminatorSEMGAN)

In [ ]:
checkpoint_SEMGAN.restore("/content/drive/MyDrive/Scriptie/ckpt/SEMGAN/SEMGAN/SemGAN-1")

In [ ]:
image_test, masked_image_test, semantics_test, semantics_masked_test, mask_test = next(iter(finetune_val.take(1)))
semantics_sparse_test = label_to_sparse(semantics_test)

In [ ]:
plt.imshow(denormalize(masked_image_test[0]))

In [ ]:
prediction = generatorSEMGAN([masked_image_test, semantics_sparse_test], training=True)
prediction = prediction*mask_test+image_test*(1-mask_test)
plt.imshow(denormalize(prediction[0]))
plt.show()

## Training SemGAN

In [ ]:
def generate_images(model, masked_input, tar, semantics_prediction, semantics_sparse, mask):
  prediction = model([masked_input, semantics_sparse], training=True)
  prediction = prediction*mask+tar*(1-mask)
  plt.figure(figsize=(15, 15))
  semantics_prediction = label_to_rgb(semantics_prediction)

  display_list = [masked_input[0], tar[0], prediction[0], semantics_prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image', 'Input Semantics']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
@tf.function
def train_epochSEMGAN(input_image, target, semantics, mask):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generatorSEMGAN([input_image, semantics], training=True)
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminatorSEMGAN([input_image, semantics, target], training=True)
    disc_generated_output = discriminatorSEMGAN([input_image, semantics, gen_output], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_L1_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generatorSEMGAN.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminatorSEMGAN.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generatorSEMGAN.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminatorSEMGAN.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss
    
  
def getValAccLossSEMGAN(input_image, target, semantics, mask): 
  gen_output = generatorSEMGAN([input_image, semantics], training=True)
  gen_output = gen_output*mask+ target*(1-mask) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminatorSEMGAN([input_image, semantics, target], training=True)
  disc_generated_output = discriminatorSEMGAN([input_image, semantics, gen_output], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_L1_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  return gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/finetune/SemGAN_GT/'
summary_writer_train = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Train")
summary_writer_val = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Val")

In [ ]:
def fitSEMGAN(train_ds, test_ds, epochs):
  image_test, masked_image_test, semantics_test, semantics_masked_test, mask_test = next(iter(test_ds.take(1)))
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    
    #semantics used for the example
    semantics_predicted_test = gen_to_label(generator_semantics(semantics_masked_test, training=True)) # turn the generator output into labels
    semantics_predicted_test = semantics_predicted_test*tf.cast(mask_test, tf.int32)+semantics_test*(1-tf.cast(mask_test, tf.int32)) # combine the prediction with the ground truth
    semantics_sparse_test = label_to_sparse(semantics_predicted_test) # make a sparse matrix as input to the generator
    #semantics_sparse_test = label_to_sparse(semantics_test)

    generate_images(generatorSEMGAN, masked_image_test, image_test, semantics_predicted_test, semantics_sparse_test, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(image, masked_image, semantics, semantics_masked, mask) in train_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True)) # turn the generator output into labels
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32)) # combine the prediction with the ground truth
      semantics_predicted = label_to_sparse(semantics_predicted) # make a sparse matrix as input to the generator
      #semantics = label_to_sparse(semantics)
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epochSEMGAN(masked_image, image, semantics_predicted, mask)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)


    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(image, masked_image, semantics, semantics_masked, mask) in test_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True))
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32))
      semantics_predicted = label_to_sparse(semantics_predicted)
      #semantics = label_to_sparse(semantics)
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLossSEMGAN(masked_image, image, semantics_predicted, mask)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 10 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)

# Fit

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
fitSEMGAN(finetune_train, finetune_val, 7)

In [ ]:
checkpoint_SEMGAN.save("/content/drive/MyDrive/Scriptie/ckpt/finetune/SEMGAN/SEMGAN-2")

#Comparison

In [ ]:
generator = Generators.Generator(OUTPUT_CHANNELS)
discriminator = Discriminators.Discriminator()

generator_semantics = Generators.GeneratorSemantics(IM_HEIGHT, IM_WIDTH)
discriminator_semantics = Discriminators.DiscriminatorSemantics(IM_HEIGHT, IM_WIDTH)

generatorSEMGAN = Generators.GeneratorSemGAN(OUTPUT_CHANNELS, IM_HEIGHT, IM_WIDTH)
discriminatorSEMGAN = Discriminators.DiscriminatorSemGAN(IM_HEIGHT, IM_WIDTH)

generatorSEMGAN_GT = Generators.GeneratorSemGAN(OUTPUT_CHANNELS, IM_HEIGHT, IM_WIDTH)
discriminatorSEMGAN_GT = Discriminators.DiscriminatorSemGAN(IM_HEIGHT, IM_WIDTH)

In [ ]:
lr = 2e-5
generator_optimizer = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.5)

In [ ]:
checkpointGAN = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

checkpointSem = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator_semantics,
                                 discriminator=discriminator_semantics)

checkpointSEMGAN = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generatorSEMGAN,
                                 discriminator=discriminatorSEMGAN)

checkpointSEMGAN_GT = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generatorSEMGAN_GT,
                                 discriminator=discriminatorSEMGAN_GT)

checkpointGAN.restore("/content/drive/MyDrive/Scriptie/ckpt/finetune/GAN/ckpt-2_2e-5")
checkpointSem.restore("/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting/finetune/ft_semantics-2")
checkpointSEMGAN.restore("/content/drive/MyDrive/Scriptie/ckpt/finetune/SEMGAN/SEMGAN-2-2")
checkpointSEMGAN_GT.restore("/content/drive/MyDrive/Scriptie/ckpt/finetune/SEMGAN_GT/SEMGAN_GT-2")

In [ ]:
image_test, masked_image_test, semantics_test, semantics_masked_test, mask_test = next(iter(finetune_val.take(1)))
sparse_semantics = label_to_sparse(semantics_test)
pred = generatorSEMGAN_GT([masked_image_test, sparse_semantics], True)
pred = pred * mask_test + image_test * (1-mask_test)
plt.imshow(pred[0]*0.5+0.5)

# Metrics

Get the metrics of the dataset. The output of *load_images_cs()* is: *image, masked_image, labels, masked_labels, mask*

In [ ]:
def metrics(image_1, image_2):
  # calculate L1 distance
  L1_distance = tf.math.reduce_sum(tf.math.abs(image_1 - image_2))
  ssim = tf.image.ssim(image_1, image_2, 2.0)
  psnr = tf.image.psnr(image_1, image_2, 2.0)
  return [L1_distance.numpy(), ssim.numpy()[0], psnr.numpy()[0]]

In [ ]:
def getIoU(label, predicted_label, mask):
  _, shape_h, shape_w, _ = label.shape
  unique_labels, _ = tf.unique(tf.reshape(label, shape_h*shape_w))
  label = tf.boolean_mask(label, mask)
  predicted_label = tf.boolean_mask(predicted_label, mask)
  IoU = tf.keras.metrics.IoU(num_classes=35, target_class_ids = unique_labels)(label, predicted_label)
  return IoU.numpy()

In [ ]:
def get_dataset_metrics(ds):
  #input_image is the inpainted image, GT is the groundtruth
  names = ["L1","SSIM","PSNR"]
  metrics_list_GAN = []
  metrics_list_SEMGAN = []
  metrics_list_SEMGAN_GT = []
  mask_list = []
  GT_list = []
  IoU_list = []
  path = "/content/predictions"

  for step, (image, masked_image, semantics, semantics_masked, mask) in enumerate(ds):
    display.clear_output(wait=True)
    prediction = generator(masked_image, training = True)
    prediction = prediction * mask + image*(1-mask)
    target_path_im = os.path.join(path, "GAN", str(step) + ".png")
    tf.keras.utils.save_img(target_path_im, prediction[0])
    
    metrics_list_GAN.append(metrics(image, prediction))
    GT_list.append(image)
    mask_list.append(mask)

    semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training = True))
    mask_int =  tf.cast(mask, tf.int32)
    semantics_predicted = semantics_predicted * mask_int + semantics * (1-mask_int)
    semantics_predicted_sparse = label_to_sparse(semantics_predicted)
    IoU = getIoU(semantics, semantics_predicted, mask)
    
    prediction = generatorSEMGAN([masked_image, semantics_predicted_sparse], training=True)
    prediction = prediction * mask + image*(1-mask)
    
    target_path_im = os.path.join(path, "SemGAN", str(step) + ".png")
    tf.keras.utils.save_img(target_path_im, prediction[0])
    target_path_sem = os.path.join(path, "SemGAN", "semantics", str(step) + ".png")
    semantics_predicted = label_to_rgb(semantics_predicted)
    tf.keras.utils.save_img(target_path_sem, semantics_predicted[0])

    metrics_list_SEMGAN.append(metrics(image, prediction))
    IoU_list.append(IoU)

    semantics_sparse = label_to_sparse(semantics)
    prediction = generatorSEMGAN_GT([masked_image, semantics_sparse], training=True) #estimated values for each of 35 classes
    prediction = prediction * mask + image*(1-mask) #insert generated label patches to inverted ground truth
    target_path_im = os.path.join(path, "SemGAN_GT", str(step) + ".png")
    tf.keras.utils.save_img(target_path_im, prediction[0])

    target_path_sem = os.path.join(path, "SemGAN_GT", "semantics", str(step) + ".png")
    semantics = label_to_rgb(semantics)
    tf.keras.utils.save_img(target_path_sem, semantics[0])
                            
    metrics_list_SEMGAN_GT.append(metrics(image, prediction))

    print(step)
    
  
  metrics_df_GAN = pd.DataFrame(metrics_list_GAN)
  metrics_df_GAN.columns = names
  metrics_df_GAN["GT"] = GT_list
  metrics_df_GAN["Mask"] = mask_list

  metrics_df_SEMGAN = pd.DataFrame(metrics_list_SEMGAN)
  metrics_df_SEMGAN.columns = names
  metrics_df_SEMGAN["IoU"] = IoU_list

  metrics_df_SEMGAN_GT = pd.DataFrame(metrics_list_SEMGAN_GT)
  metrics_df_SEMGAN_GT.columns = names

  return metrics_df_GAN, metrics_df_SEMGAN, metrics_df_SEMGAN_GT

In [ ]:
! mkdir /content/predictions
! mkdir /content/predictions/GAN
! mkdir /content/predictions/SemGAN
! mkdir /content/predictions/SemGAN_GT
! mkdir /content/predictions/SemGAN/semantics
! mkdir /content/predictions/SemGAN_GT/semantics

In [ ]:
metrics_ft_GAN, metrics_ft_SEMGAN, metrics_ft_SEMGAN_GT = get_dataset_metrics(finetune_val)

In [ ]:
metrics_ft_SEMGAN["IoU"].mean()

In [ ]:
print(metrics_ft_GAN["L1"].mean())
print(metrics_ft_GAN["PSNR"].mean())
print(metrics_ft_GAN["SSIM"].mean())
print(metrics_ft_GAN["L1"].std())
print(metrics_ft_GAN["PSNR"].std())
print(metrics_ft_GAN["SSIM"].std())

In [ ]:
print(metrics_ft_SEMGAN["L1"].mean())
print(metrics_ft_SEMGAN["PSNR"].mean())
print(metrics_ft_SEMGAN["SSIM"].mean())
print(metrics_ft_SEMGAN["L1"].std())
print(metrics_ft_SEMGAN["PSNR"].std())
print(metrics_ft_SEMGAN["SSIM"].std())

In [ ]:
print(metrics_ft_SEMGAN_GT["L1"].mean())
print(metrics_ft_SEMGAN_GT["PSNR"].mean())
print(metrics_ft_SEMGAN_GT["SSIM"].mean())
print(metrics_ft_SEMGAN_GT["L1"].std())
print(metrics_ft_SEMGAN_GT["PSNR"].std())
print(metrics_ft_SEMGAN_GT["SSIM"].std())

In [ ]:
metrics_ft_SEMGAN[["L1", "PSNR", "SSIM"]]

In [ ]:
metrics_ft_SEMGAN

In [ ]:
metrics_ft_SEMGAN_GT

# Visualize the images

In [ ]:
im = load("/content/predictions/GAN/1.png", 3)
plt.imshow(normalize(im)*0.5+0.5)

In [ ]:
def visualize_rows(row):
  path = "/content/predictions"
  GT = metrics_ft_GAN.iloc[row]["GT"]
  mask = metrics_ft_GAN.iloc[row]["Mask"]
  inp_im = tf.where(mask == 1, -1, GT)
  

  GAN_pred = normalize(load(os.path.join(path, "GAN", str(row)+".png"), 3))
  SEMGAN_pred = normalize(load(os.path.join(path, "SemGAN", str(row)+".png"), 3))
  SEMGAN_GT_pred = normalize(load(os.path.join(path, "SemGAN_GT", str(row)+".png"), 3))

  semantics_pred = load(os.path.join(path, "SemGAN", "semantics", str(row)+".png"), 3)
  semantics_pred = normalize(semantics_pred)
  semantics = load(os.path.join(path, "SemGAN_GT", "semantics", str(row)+".png"), 3)

  fix, axs = plt.subplots(6,1, figsize = (20,20))
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  
  images = [inp_im[0], semantics_pred, GT[0], GAN_pred, SEMGAN_pred, SEMGAN_GT_pred]
  for i, im in enumerate(images):
    axs[i].imshow(im*0.5+0.5)
    axs[i].axis("off")



In [ ]:
visualize_rows(0)

In [ ]:
def pd_visualize_rows(ds, rows):
  n = len(rows)
  fix, axs = plt.subplots(n,3, figsize = (20,20))
  plt.subplots_adjust(wspace=0, hspace=0)

  for i, row in enumerate(rows):
    l1, ssim, psnr, image, semantics, mask = ds.iloc[row]
    masked_image = tf.where(mask == 1, -1, image)
    prediction = generator(masked_image, training = True)
    prediction = prediction * mask + image * (1-mask)

    if n > 1:
      axs[i][0].imshow(denormalize(image[0]))
      axs[i][1].imshow(denormalize(masked_image[0]))
      axs[i][2].imshow(denormalize(prediction[0]))
      for ax in axs[i]:
          ax.axis("off")
    else:
      axs[0].imshow(denormalize(image[0]))
      axs[1].imshow(denormalize(masked_image[0]))
      axs[2].imshow(denormalize(prediction[0]))

      for ax in axs:
          ax.axis("off")
      
  plt.tight_layout()
  plt.show()

In [ ]:
def compare_images():
  path = "/content/predictions"
  